# Creating a Cutout with the SARAH-2 dataset

This walkthrough describes the process of creating a cutout using the [SARAH-2 dataset by EUMETSAT](https://wui.cmsaf.eu/safira/action/viewDoiDetails?acronym=SARAH_V002_01).

The SARAH-2 dataset contains extensive information on solar radiation variables, like surface incoming direct radiation (SID) or surface incoming shortwave radiation (SIS).
It serves as an addition to the ERA5 dataset and as such requires the `cdsapi` to be setup properly.

> **Recommendation**
>
> This is a reduced version for cutout creation. Creating cutouts with ERA-5 is simpler and explained in more details.
> We therefore recommend you have a look at [this example first](https://atlite.readthedocs.io/en/latest/examples/create_cutout.html).

> **Note**:
>
> For creating a cutout from this dataset, you need to download large files and your computers memory needs to be able to handle these as well.

## Downloading the data set

To download the dataset, head to the EUMETSTATs website (the link points to the current 2.1 edition)

https://wui.cmsaf.eu/safira/action/viewDoiDetails?acronym=SARAH_V002_01 

On the bottom, select the products you want to include in the cutout, i.e. for us:

| variable | time span | time resolution | 
| --- | --- | --- |
| Surface incoming direct radiation (SID) | 2013 | Instantaneous |
| Surface incoming shortwave radiation (SIS) | 2013 | Instantaneous |

* Add each product to your cart and register with the website.
* Follow the instructions to activate your account, confirm your order and wait for the download to be ready.
* You will be notified by email with the download instructions.
* Download the ordered files of your order into a directory, e.g. `sarah-2`.
* Extract the `tar` files (e.g. for linux systems `tar -xvf *` or with `7zip` for windows) into the same folder

You are now ready to create cutouts using the SARAH-2 dataset.

## Specifying the cutout

Import the package and set recommended logging settings:

In [7]:
import logging 

import atlite

logging.basicConfig(level=logging.INFO)

In [8]:
import xarray as xr

In [ ]:
#ds=xr.open_dataset("C:/Users/marta/Desktop/Thesis/Climate-Change-Impacted-Solar-Energy-Generation/atlite_examples/sarah_data/SIDin202501010000004UD10001I1UD.nc")

In [9]:
start_date="2025-01-01"
end_date="2025-01-02"
cutout = atlite.Cutout(
    path="europe-202501",
    module=["sarah", "era5"],
    sarah_dir="C:/Users/marta/Desktop/Thesis/Climate-Change-Impacted-Solar-Energy-Generation/atlite_examples/sarah_data",
    x=slice(-13.6913, 1.7712),
    y=slice(49.9096, 60.8479),
    time=slice(start_date, end_date),
    chunks={"time": 100},
)
print(cutout.data)

<xarray.Dataset> Size: 9MB
Dimensions:               (x: 62, y: 44, time: 48)
Coordinates:
  * x                     (x) float64 496B -13.5 -13.25 -13.0 ... 1.25 1.5 1.75
  * y                     (y) float64 352B 50.0 50.25 50.5 ... 60.25 60.5 60.75
  * time                  (time) datetime64[ns] 384B 2025-01-01 ... 2025-01-0...
    lon                   (x) float64 496B dask.array<chunksize=(62,), meta=np.ndarray>
    lat                   (y) float64 352B dask.array<chunksize=(44,), meta=np.ndarray>
Data variables: (12/15)
    influx_direct         (time, y, x) float32 524kB dask.array<chunksize=(48, 44, 62), meta=np.ndarray>
    influx_diffuse        (time, y, x) float32 524kB dask.array<chunksize=(48, 44, 62), meta=np.ndarray>
    solar_altitude        (time, y, x) float64 1MB dask.array<chunksize=(48, 44, 62), meta=np.ndarray>
    solar_azimuth         (time, y, x) float64 1MB dask.array<chunksize=(48, 44, 62), meta=np.ndarray>
    height                (y, x) float32 11kB dask.a

c:\Users\marta\anaconda3\envs\atlite_cmip6_2025\Lib\site-packages\atlite\cutout.py:156: UserWarning: Arguments module, sarah_dir, x, y, time are ignored, since cutout is already built.
  warn(


In [8]:
cutout

<Cutout "europe-202501">
 x = -13.50 ⟷ 1.75, dx = 0.25
 y = 50.00 ⟷ 60.75, dy = 0.25
 time = 2025-01-01 ⟷ 2025-01-02, dt = h
 module = ['sarah', 'era5']
 prepared_features = []

Let's see what the available features that is the available weather data variables are.

In [9]:
cutout.available_features.to_frame()

variables
module feature                          
sarah  influx              influx_direct
       influx             influx_diffuse
       influx             solar_altitude
       influx              solar_azimuth
era5   height                     height
       wind                      wnd100m
       wind                wnd_shear_exp
       wind                  wnd_azimuth
       wind                    roughness
       influx                 influx_toa
       influx              influx_direct
       influx             influx_diffuse
       influx                     albedo
       influx             solar_altitude
       influx              solar_azimuth
       temperature           temperature
       temperature      soil temperature
       temperature  dewpoint temperature
       runoff                     runoff

## Preparing the Cutout

No matter which dataset you use, this is where all the work actually happens.
This can be fast or take some or a lot of time and resources, among others depending on
your computer ressources (especially memory for SARAH-2).

In [10]:
cutout.prepare()

INFO:atlite.data:Storing temporary files in C:\Users\marta\AppData\Local\Temp\tmpn0rcka5w
INFO:atlite.data:Calculating and writing with module sarah:


c:\Users\marta\anaconda3\envs\atlite_cmip6_2025\Lib\site-packages\atlite\pv\solar_position.py:71: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  time_shift = pd.to_timedelta(time_shift)
INFO:atlite.data:Calculating and writing with module era5:
INFO:atlite.datasets.era5:Requesting data for feature height...
INFO:atlite.datasets.era5:Requesting data for feature temperature...
INFO:atlite.datasets.era5:Requesting data for feature runoff...
INFO:atlite.datasets.era5:Requesting data for feature wind...
INFO:atlite.datasets.era5:Requesting data for feature influx...
2025-02-10 11:39:19,131 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-02-10 11:39:19,131 WARNING [2024-06-16T00:00:00] CDS API syntax is 

<Cutout "europe-202501">
 x = -13.50 ⟷ 1.75, dx = 0.25
 y = 50.00 ⟷ 60.75, dy = 0.25
 time = 2025-01-01 ⟷ 2025-01-02, dt = h
 module = ['sarah', 'era5']
 prepared_features = ['influx', 'height', 'wind', 'temperature', 'runoff']

Querying the cutout gives us basic information on which data is contained and can already be used.

## Inspecting the Cutout

In [11]:
cutout  # basic information

<Cutout "europe-202501">
 x = -13.50 ⟷ 1.75, dx = 0.25
 y = 50.00 ⟷ 60.75, dy = 0.25
 time = 2025-01-01 ⟷ 2025-01-02, dt = h
 module = ['sarah', 'era5']
 prepared_features = ['influx', 'height', 'wind', 'temperature', 'runoff']

In [12]:
cutout.data.attrs  # cutout meta data

{'module': ['sarah', 'era5'],
 'prepared_features': ['runoff', 'influx', 'temperature', 'height', 'wind'],
 'chunksize_time': np.int64(100),
 'sarah_dir': 'C:/Users/marta/Desktop/Thesis/Climate-Change-Impacted-Solar-Energy-Generation/atlite_examples/sarah_data',
 'CDI': 'Climate Data Interface version 2.4.0 (https://mpimet.mpg.de/cdi)',
 'Conventions': 'CF-1.7',
 'institution': 'European Centre for Medium-Range Weather Forecasts',
 'title': 'CM SAF Surface Solar Radiation Climate Data Record using Heliosat, Edition 3 (SARAH-3)',
 'summary': 'This file contains data from the CM SAF Surface Solar Radiation Interim Climate Data Record using Heliosat, Edition 3',
 'id': 'DOI:10.5676/EUM_SAF_CM/SARAH/V003',
 'product_version': '3.0',
 'creator_name': 'DE/DWD',
 'creator_email': 'contact.cmsaf@dwd.de',
 'creator_url': 'http://www.cmsaf.eu/',
 'creator_type': 'institution',
 'publisher_name': 'EUMETSAT/CMSAF',
 'publisher_email': 'contact.cmsaf@dwd.de',
 'publisher_url': 'http://www.cmsaf.eu/

In [13]:
cutout.prepared_features  # included weather variables

module  feature    
sarah   influx                influx_direct
        influx               influx_diffuse
        influx               solar_altitude
        influx                solar_azimuth
era5    height                       height
        wind                        wnd100m
        wind                  wnd_shear_exp
        wind                    wnd_azimuth
        wind                      roughness
        influx                   influx_toa
        influx                       albedo
        temperature             temperature
        temperature        soil temperature
        temperature    dewpoint temperature
        runoff                       runoff
dtype: object

In [14]:
cutout.data  # access to underlying xarray data

<xarray.Dataset> Size: 9MB
Dimensions:               (x: 62, y: 44, time: 48)
Coordinates:
  * x                     (x) float64 496B -13.5 -13.25 -13.0 ... 1.25 1.5 1.75
  * y                     (y) float64 352B 50.0 50.25 50.5 ... 60.25 60.5 60.75
  * time                  (time) datetime64[ns] 384B 2025-01-01 ... 2025-01-0...
    lon                   (x) float64 496B dask.array<chunksize=(62,), meta=np.ndarray>
    lat                   (y) float64 352B dask.array<chunksize=(44,), meta=np.ndarray>
Data variables: (12/15)
    influx_direct         (time, y, x) float32 524kB dask.array<chunksize=(48, 44, 62), meta=np.ndarray>
    influx_diffuse        (time, y, x) float32 524kB dask.array<chunksize=(48, 44, 62), meta=np.ndarray>
    solar_altitude        (time, y, x) float64 1MB dask.array<chunksize=(48, 44, 62), meta=np.ndarray>
    solar_azimuth         (time, y, x) float64 1MB dask.array<chunksize=(48, 44, 62), meta=np.ndarray>
    height                (y, x) float32 11kB dask.array<chunksize=(44, 62), meta=np.ndarray>
    wnd100m               (time, y, x) float32 524kB dask.array<chunksize=(48, 44, 62), meta=np.ndarray>
    ...                    ...
    influx_toa            (time, y, x) float32 524kB dask.array<chunksize=(48, 44, 62), meta=np.ndarray>
    albedo                (time, y, x) float32 524kB dask.array<chunksize=(48, 44, 62), meta=np.ndarray>
    temperature           (time, y, x) float32 524kB dask.array<chunksize=(48, 44, 62), meta=np.ndarray>
    soil temperature      (time, y, x) float32 524kB dask.array<chunksize=(48, 44, 62), meta=np.ndarray>
    dewpoint temperature  (time, y, x) float32 524kB dask.array<chunksize=(48, 44, 62), meta=np.ndarray>
    runoff                (time, y, x) float32 524kB dask.array<chunksize=(48, 44, 62), meta=np.ndarray>
Attributes: (12/48)
    module:                     ['sarah', 'era5']
    prepared_features:          ['runoff', 'influx', 'temperature', 'height',...
    chunksize_time:             100
    sarah_dir:                  C:/Users/marta/Desktop/Thesis/Climate-Change-...
    CDI:                        Climate Data Interface version 2.4.0 (https:/...
    Conventions:                CF-1.7
    ...                         ...
    license:                    The CM SAF data are owned by EUMETSAT and are...
    history:                    2025-02-10T10:39 GRIB to CDM+CF via cfgrib-0....
    CDO:                        Climate Data Operators version 2.4.0 (https:/...
    GRIB_centre:                ecmf
    GRIB_centreDescription:     European Centre for Medium-Range Weather Fore...
    GRIB_subCentre:             0